In [ ]:
import sys, json
sys.path.insert(1, '../data_collection')
from collection import *

import pandas as pd
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots
import plotly.express as px

In [ ]:
df_gtemp = pd.read_csv('global_temp.csv')
df_gtemp['date'] = pd.to_datetime(df_gtemp['date'])

with open('../data_collection/countries.json', 'r') as f:
    countries_json = json.load(f)
country_alpha3 = [[country, countries_json[country]['alpha3']] for country in countries_json.keys()]
df_country_alpha3 = pd.DataFrame(country_alpha3, columns = ['country', 'alpha3'])
df_gtemp = df_gtemp.merge(df_country_alpha3, on = 'country')

df_gtemp = df_gtemp[df_gtemp.date.dt.year > 1849]
df_gtemp['yearly_mean'] = df_gtemp.groupby(by = ['country', df_gtemp.date.dt.year])['monthly_anomaly'].transform('mean')
df_gtemp_yearly = df_gtemp.groupby(by = ['country', df_gtemp.date.dt.year]).first().drop(columns = ['date', 'monthly_anomaly']).reset_index()
yearly_min = df_gtemp_yearly.loc[:, 'yearly_mean'].min()
yearly_max = df_gtemp_yearly.loc[:, 'yearly_mean'].max()
df_gtemp_yearly

In [ ]:
print('min: ', yearly_min)
print('max: ', yearly_max)

In [ ]:
fig = px.choropleth(data_frame = df_gtemp_yearly,
                    locations = 'alpha3',
                    color = 'yearly_mean',  # value in column 'Confirmed' determines color
                    hover_name = 'country',
                    color_continuous_scale = 'Jet',  #  color scale red, yellow green
                    range_color = [yearly_min, yearly_max],
                    animation_frame = 'date')

fig.show()

In [ ]:
afg = df_gtemp[df_gtemp.loc[:, 'country'] == 'Afghanistan']
fig = go.Figure(data=go.Scatter(x = afg['date'], y = afg['monthly_anomaly']))
fig.show()